In [5]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def H4(n,m): #bad definition
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
 #if argument is an S type then 'polynomial' takes no argument.
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

In [4]:
j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

In [4]:
print(H6(5,3));print(type(H6(5,3)))

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1
<type 'sage.rings.polynomial.polynomial_rational_flint.Polynomial_rational_flint'>


In [6]:
H6(5,4)

-10569646080*x^5 - 193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1

In [7]:
H6(5,3)
    

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1

In [6]:
# correct signs, at the expense of deviating from the
# normalization in the 4mar21 draft.
poly=H6(5,3)
plist=poly.list()
for k in [1..5]:
    print [k,-504*sigma(k,5),plist[k]]

[1, -504, -504]
[2, -16632, -16632]
[3, -122976, -122976]
[4, -532728, -532728]
[5, -1575504, -1575504]


In [4]:
diffs=[]
poly=H6(100,3)
plist=poly.list()
for k in [1..poly.degree()]:
    diffs=diffs+[-504*sigma(k,5)-plist[k]]
print diffs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
import pickle
wfile = open('/Users/barrybrent/7mar21no2.txt','a')
data=[]
for m in [3..9]:
    if 1 in divisors(m):
        print(m)
    poly=H6(12,m)
    data=data+[[m,poly.list()]]
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

3
4
5
6
7
8
9


In [12]:
import pickle
rfile = open('/Users/barrybrent/7mar21no2.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(s)

[[3, [1, -504, -16632, -122976, -532728, -1575504, -4058208, -8471232, -17047800, -29883672, -51991632, -81170208, -129985632]], [4, [1, -1280, -102400, -9175040, -193986560, -10569646080, -187904819200, -7387343749120, -100674033418240, -4161651511132160, -55415386039910400, -1874623344892968960, -23328646069779169280]], [5, [1, -2600, -351000, -276380000/3, -29091725000/3, -461573190000, -884466669580000/9, -42731073946600000/9, -2925509932307625000/7, -22528177776031841875000/567, -764196116461702063750000/567, -4519028573198488719500000/21, -20593777488239481724217500000/1701]], [6, [1, -4608, -884736, -528482304, -125627793408, -14843406974976, -3087428650795008, -1013309916158361600, -66148871326817845248, -20254524992933087674368, -3060093480899530098475008, -326409971295949877170667520, -63367055760920402821398921216]], [7, [1, -7448, -1824760, -2226207200, -932746146360, -1088046362821712/5, -281820099671626208/5, -36817178522254305728, -9662291816318397239480, -12139508201766

In [3]:
# Had to shut down & restart to avoid an error message
# bc two looping routines involving H6 throws one..
import pickle
wfile = open('/Users/barrybrent/7mar21no3.txt','a')
data=[]
for m in [3..9]:
    if 1 in divisors(m):
        print(m)
    poly=H6(12,m)
    data=data+[[m,poly]]
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

3
4
5
6
7
8
9


In [4]:
import pickle
rfile = open('/Users/barrybrent/7mar21no3.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(s)

[[3, -129985632*x^12 - 81170208*x^11 - 51991632*x^10 - 29883672*x^9 - 17047800*x^8 - 8471232*x^7 - 4058208*x^6 - 1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1], [4, -23328646069779169280*x^12 - 1874623344892968960*x^11 - 55415386039910400*x^10 - 4161651511132160*x^9 - 100674033418240*x^8 - 7387343749120*x^7 - 187904819200*x^6 - 10569646080*x^5 - 193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1], [5, -20593777488239481724217500000/1701*x^12 - 4519028573198488719500000/21*x^11 - 764196116461702063750000/567*x^10 - 22528177776031841875000/567*x^9 - 2925509932307625000/7*x^8 - 42731073946600000/9*x^7 - 884466669580000/9*x^6 - 461573190000*x^5 - 29091725000/3*x^4 - 276380000/3*x^3 - 351000*x^2 - 2600*x + 1], [6, -63367055760920402821398921216*x^12 - 326409971295949877170667520*x^11 - 3060093480899530098475008*x^10 - 20254524992933087674368*x^9 - 66148871326817845248*x^8 - 1013309916158361600*x^7 - 3087428650795008*x^6 - 14843406974976*x^5 - 125627793408*x^4 - 528482304*

In [18]:
data=[]
for m in [3..10]:
    print m
    data=data+[[m,H6(5,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no11.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

3
4
5
6
7
8
9
10


In [20]:
import pickle
rfile = open('/Users/barrybrent/8mar21no11.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in  [0..3]:
    print s[k][1]
    print "--------------------------------------------------------------------------------------"

-1575504*x^5 - 532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1
--------------------------------------------------------------------------------------
-10569646080*x^5 - 193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1
--------------------------------------------------------------------------------------
-461573190000*x^5 - 29091725000/3*x^4 - 276380000/3*x^3 - 351000*x^2 - 2600*x + 1
--------------------------------------------------------------------------------------
-14843406974976*x^5 - 125627793408*x^4 - 528482304*x^3 - 884736*x^2 - 4608*x + 1
--------------------------------------------------------------------------------------


In [2]:
data=[]
for m in [3..400]:
    if 10 in divisors(m):
        print m
    data=data+[[m,H6(100,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no13.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

10
20
30
40
50
60
70
80
90
100
110
120


KeyboardInterrupt: 

In [3]:
len(data)

118

In [5]:
import pickle
rfile = open('/Users/barrybrent/16mar21no13.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for k in [2..2]:
    m = s[k][0]
    poly=D(s[k][1])
    print(poly)
    print(poly.degree())

-4509761294051827662634974954300352219867608087127729347058355562482339053873913601618326183599905898068057789167461648441704267267156243924537964971450154371474087612180110462438348241643731525734948816805777690941425817072740755975246429443359375000/168408038462625364828864964197173846165392820165767333654878925721231*x^100 - 939574190575591953280248432803410626887563854330249770485734978761147947904420418933296150967498426718043420307926706883539813969577307624884344691626055165128478498936337646659090799778220314348005013743306797492151350076019298285245895385742187500000/4563741252156673261609666700152813187650184769924487672354688586250630017*x^99 - 4712485003779872167121960552518128382863775102298301920335870651196848467420869620201848043110053781291218531910897147113007269710249240297752117342184696161958402359416211015239400849751449375044519031880146940238773822784423828125000/565127055492208306479855653510156954200726939415477455419*x^98 - 14773257386391580390548977022281328

In [4]:
import pickle
rfile = open('/Users/barrybrent/8mar21no13.txt','r')
data=[]
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
lnth=len(s)
s[lnth-1][0]

120

In [2]:
import pickle
wfile = open('/Users/barrybrent/9mar21no3.txt','w')
data=[]
for m in [121..400]:
    print(m)
    data=data+[[m,H6(100,m)]]
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370


In [13]:
sfile = open('/Users/barrybrent/8mar21no13.txt','r')
rs=pickle.load(sfile)
sfile.close()
s=stripQuotationMarks(rs)

import pickle
tfile = open('/Users/barrybrent/9mar21no3.txt','r')
rt=pickle.load(tfile)
tfile.close()
t=stripQuotationMarks(rt)

v=s+t
print([len(s),len(t),len(v)])

[118, 280, 398]


In [15]:
sfile = open('/Users/barrybrent/8mar21no13.txt','r')
rs=pickle.load(sfile)
sfile.close()
s=stripQuotationMarks(rs)

import pickle
tfile = open('/Users/barrybrent/9mar21no3.txt','r')
rt=pickle.load(tfile)
tfile.close()
t=stripQuotationMarks(rt)

v=s+t

import pickle
wfile = open('/Users/barrybrent/16mar21no13.txt','w')
pv = pickle.dumps(str(v))
wfile.write(pv)
wfile.close()

In [2]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/16mar21no13.txt','r') # H6 series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    m = s[k][0]
    poly=D(s[k][1])
    data=data+[[k,poly.degree()]]
print(data) # correctness check: all of the series have the right degree.

[[0, 100], [1, 100], [2, 100], [3, 100], [4, 100], [5, 100], [6, 100], [7, 100], [8, 100], [9, 100], [10, 100], [11, 100], [12, 100], [13, 100], [14, 100], [15, 100], [16, 100], [17, 100], [18, 100], [19, 100], [20, 100], [21, 100], [22, 100], [23, 100], [24, 100], [25, 100], [26, 100], [27, 100], [28, 100], [29, 100], [30, 100], [31, 100], [32, 100], [33, 100], [34, 100], [35, 100], [36, 100], [37, 100], [38, 100], [39, 100], [40, 100], [41, 100], [42, 100], [43, 100], [44, 100], [45, 100], [46, 100], [47, 100], [48, 100], [49, 100], [50, 100], [51, 100], [52, 100], [53, 100], [54, 100], [55, 100], [56, 100], [57, 100], [58, 100], [59, 100], [60, 100], [61, 100], [62, 100], [63, 100], [64, 100], [65, 100], [66, 100], [67, 100], [68, 100], [69, 100], [70, 100], [71, 100], [72, 100], [73, 100], [74, 100], [75, 100], [76, 100], [77, 100], [78, 100], [79, 100], [80, 100], [81, 100], [82, 100], [83, 100], [84, 100], [85, 100], [86, 100], [87, 100], [88, 100], [89, 100], [90, 100], [91, 100

In [16]:
import pickle
rfile = open('/Users/barrybrent/16mar21no13.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/16mar21no14.txt','w')
enoughfile=[]
for qpower in [0..100]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1] 
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower,rl]] 
    finish = time.time()
    print([qpower,finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 1.6308491230010986]
[1, 1.1170260906219482]
[2, 1.0831561088562012]
[3, 1.1159141063690186]
[4, 1.1132071018218994]
[5, 1.121748924255371]
[6, 1.1123900413513184]
[7, 1.1246509552001953]
[8, 1.1100549697875977]
[9, 1.120030164718628]
[10, 1.1243479251861572]
[11, 1.0846130847930908]
[12, 1.1325738430023193]
[13, 1.1253068447113037]
[14, 1.1254000663757324]
[15, 1.1348769664764404]
[16, 1.1515848636627197]
[17, 1.1625990867614746]
[18, 1.1765599250793457]
[19, 1.1864590644836426]
[20, 1.1133451461791992]
[21, 1.1472880840301514]
[22, 1.1520378589630127]
[23, 1.165959119796753]
[24, 1.1609439849853516]
[25, 1.1733391284942627]
[26, 1.1751060485839844]
[27, 1.1848740577697754]
[28, 1.1893329620361328]
[29, 1.147947072982788]
[30, 1.187859058380127]
[31, 1.234184980392456]
[32, 1.2353529930114746]
[33, 1.2241530418395996]
[34, 1.2374320030212402]
[35, 1.2255620956420898]
[36, 1.2609128952026367]
[37, 1.2560858726501465]
[38, 1.2294819355010986]
[39, 1.2569329738616943]
[40, 1.280055999

In [3]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/16mar21no14.txt','r') # H6 interpolating poly series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    m = s[k][0]
    poly=D(s[k][1])
    data=data+[[k,poly.degree()]]
print(data) # correctness check: all of the series have the right degree.

[[0, 0], [1, 3], [2, 6], [3, 9], [4, 12], [5, 15], [6, 18], [7, 21], [8, 24], [9, 27], [10, 30], [11, 33], [12, 36], [13, 39], [14, 42], [15, 45], [16, 48], [17, 51], [18, 54], [19, 57], [20, 60], [21, 63], [22, 66], [23, 69], [24, 72], [25, 75], [26, 78], [27, 81], [28, 84], [29, 87], [30, 90], [31, 93], [32, 96], [33, 99], [34, 102], [35, 105], [36, 108], [37, 111], [38, 114], [39, 117], [40, 120], [41, 123], [42, 126], [43, 129], [44, 132], [45, 135], [46, 138], [47, 141], [48, 144], [49, 147], [50, 150], [51, 153], [52, 156], [53, 159], [54, 162], [55, 165], [56, 168], [57, 171], [58, 174], [59, 177], [60, 180], [61, 183], [62, 186], [63, 189], [64, 192], [65, 195], [66, 198], [67, 201], [68, 204], [69, 207], [70, 210], [71, 213], [72, 216], [73, 219], [74, 222], [75, 225], [76, 228], [77, 231], [78, 234], [79, 237], [80, 240], [81, 243], [82, 246], [83, 249], [84, 252], [85, 255], [86, 258], [87, 261], [88, 264], [89, 267], [90, 270], [91, 273], [92, 276], [93, 279], [94, 282], [9

In [17]:
import pickle
rfile = open('/Users/barrybrent/16mar21no14.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

101

In [18]:
s[0]

[0, 1]

In [19]:
s[1]

[1, -24*x^3 + 16*x^2]

In [20]:
s[2]

[2, 24*x^6 - 400*x^5 + 928*x^4 - 448*x^3]

In [ ]:
# Following cells indicate that the SageMath file essentially agrees with the Mathematica file.
# For me this was a question, because of my use of the Mathematica "Refine" command.

In [23]:
import pickle
rtfile = open('/Users/barrybrent/run6sept20no10','r')
# Mathematica file
L=rtfile.readlines()
rtfile.close()
for n in [0..29]:
    print(L[n])

{3, 1 - 504*x - 16632*x^2 - 122976*x^3 - 532728*x^4 - 1575504*x^5 - 

  4058208*x^6 - 8471232*x^7 - 17047800*x^8 - 29883672*x^9 - 51991632*x^10 - 

  81170208*x^11 - 129985632*x^12 - 187132176*x^13 - 279550656*x^14 - 

  384422976*x^15 - 545530104*x^16 - 715608432*x^17 - 986161176*x^18 - 

  1247954400*x^19 - 1665307728*x^20 - 2066980608*x^21 - 2678616864*x^22 - 

  3243917376*x^23 - 4159663200*x^24 - 4923450504*x^25 - 6175361808*x^26 - 

  7261732800*x^27 - 8954092224*x^28 - 10337619600*x^29 - 12685958208*x^30 - 

  14429092608*x^31 - 17456963832*x^32 - 19805530752*x^33 - 23615078256*x^34 - 

  26481071232*x^35 - 31587041304*x^36 - 34949354832*x^37 - 41182495200*x^38 - 

  45660250944*x^39 - 53291422800*x^40 - 58391525808*x^41 - 68210360064*x^42 - 

  74092255776*x^43 - 85796909856*x^44 - 93416358672*x^45 - 

  107049273408*x^46 - 115589884032*x^47 - 133109345376*x^48 - 

  142375996728*x^49 - 162473866632*x^50 - 174608457408*x^51 - 

  197798710032*x^52 - 210770528976*x^53 - 23963718

In [5]:
import pickle
rsfile = open('/Users/barrybrent/8mar21no13.txt','r')
# SageMath file
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
s[0]

[3,
 -5204087182728*x^100 - 4812825142944*x^99 - 4698407892024*x^98 - 4328019490032*x^97 - 4259499175008*x^96 - 3901105454400*x^95 - 3814466173056*x^94 - 3520698596352*x^93 - 3428820666432*x^92 - 3145317614208*x^91 - 3082739836176*x^90 - 2814365962800*x^89 - 2745582285600*x^88 - 2522379182400*x^87 - 2445044440608*x^86 - 2236991958432*x^85 - 2184798502656*x^84 - 1985276484576*x^83 - 1926920351664*x^82 - 1764601070904*x^81 - 1705327105104*x^80 - 1550836425600*x^79 - 1506788281152*x^78 - 1364308856064*x^77 - 1319087800800*x^76 - 1201321922976*x^75 - 1153328709456*x^74 - 1044828083376*x^73 - 1010815205400*x^72 - 909331593408*x^71 - 873875350656*x^70 - 791515839744*x^69 - 756398112624*x^68 - 680463054432*x^67 - 653582514816*x^66 - 584975182176*x^65 - 558622843128*x^64 - 502284758976*x^63 - 476160056064*x^62 - 425676536208*x^61 - 406335085632*x^60 - 360321847200*x^59 - 341141446800*x^58 - 304500873600*x^57 - 286539422400*x^56 - 253738070208*x^55 - 239637182400*x^54 - 210770528976*x^53 - 1977

In [35]:
import pickle
rtfile = open('/Users/barrybrent/run6sept20no10','r')
# Mathematica file
L=rtfile.readlines()
rtfile.close()
for n in [30..147]:
    print(L[n])

{4, 1 - 1280*x - 102400*x^2 - 9175040*x^3 - 193986560*x^4 - 10569646080*x^5 - 

  187904819200*x^6 - 7387343749120*x^7 - 100674033418240*x^8 - 

  4161651511132160*x^9 - 55415386039910400*x^10 - 1874623344892968960*x^11 - 

  23328646069779169280*x^12 - 791912958476828016640*x^13 - 

  9915124939618883993600*x^14 - 325400565460236490506240*x^15 - 

  3454706230124324826644480*x^16 - 116028544484107182900510720*x^17 - 

  1429932571012928570707148800*x^18 - 41466155612781780692421836800*x^19 - 

  450880973683472096998415400960*x^20 - 14904798072995978509785455656960*

   x^21 - 164893613232812652616563346636800*x^22 - 

  4820241407367842299191213938442240*x^23 - 

  51999027621361970049394765484523520*x^24 - 

  1597341168335587304237965775030190080*x^25 - 

  17832294523530548636734423330665267200*x^26 - 

  530652738942259383635816724846293811200*x^27 - 

  5287004353234502884474892582265070551040*x^28 - 

  162099354085970490700619271001188021043200*x^29 - 

  187580654765167327984